In [5]:
import matplotlib
matplotlib.matplotlib_fname()
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [6]:
matplotlib.font_manager._rebuild()

In [1]:
import progressbar

In [3]:
def say_my_name(name):
    """
    Print the current widget value in short sentence
    """
    print(f'My name is {name}')
     
widgets.interact(say_my_name, name=["Jim", "Emma", "Bond"]);

interactive(children=(Dropdown(description='name', options=('Jim', 'Emma', 'Bond'), value='Jim'), Output()), _…

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.common.exceptions import NoSuchElementException   
import progressbar

options = Options()
options.headless = True
options.add_argument("--window-size=1920,10200") #this is important, to tell it how much of the webpage to import
driver = webdriver.Chrome(options=options, executable_path=r'/usr/local/bin/chromedriver')


In [10]:
from __future__ import print_function

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.common.exceptions import NoSuchElementException   
import progressbar


import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [11]:
def rose_complete():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    
    # The ID and range of a sample spreadsheet.
    sheetId = '1uxUNBIjPzIJznquQ1rtHEiWE8iq_TNsJ3m9x8AAO0n0'
    SAMPLE_SPREADSHEET_ID_input = sheetId #'1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
    SAMPLE_RANGE_NAME = 'ROSE!A1:P26'
    
    #global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'my_json_file.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')
        
    df=pd.DataFrame(values_input[1:], columns=values_input[0])
    
    return df

In [12]:
def scarica_stats(stagione ='2020-21'):
    data = pd.read_excel('http://www.fantacalcio.it/Servizi/Excel.ashx?type=2&r=1&t=1604259046000&s='+stagione,skiprows = [0])
    data = data[data.Nome != 'Nome']
    data = data.dropna()
    data.index = list(range(len(data)))
    return data

In [13]:
def scarica_quot(stagione ='2020-21'):
    data = pd.read_excel('http://www.fantacalcio.it//Servizi/Excel.ashx?type=0&r=1&t=1604293589000',skiprows = [0])
    #data = data[data.Nome != 'Nome']
    #data = data.dropna()
    data.index = list(range(len(data)))
    return data

In [14]:
def personal_info(Name, database):
    gioc = database[database.Nome == Name]
    name = Name.replace(' ','-').lower()
    Id = str(list(gioc['Id'])[0])
    team = list(gioc['Squadra'])[0].lower()
    link = 'https://www.fantacalcio.it/squadre/'+team+'/'+name+'/'+Id
    driver.get(link)
    string = driver.find_element_by_xpath("/html/body/div[7]/div[5]/main/div/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/ul[1]/li[3]")
    classe = string.text[16:26]
    eta = string.text[28:30]
    
    return classe, eta

In [18]:
dict_names={
    'AS 800A': 'enzo',
    'PDG 1908': 'pietro',
    'IGNORANZA EVERYWHERE': 'mario',
    'SOROS FC': 'musci8',
    'MAINZ NA GIOIA': 'franky',
    'PALLA PAZZA': 'nanni',
    'I DISEREDATI': 'emiliano',
    'XYZ': 'luca',
    'AS 800A - PRIMAVERA': 'enzo',
    'PDG 1908 - PRIMAVERA': 'pietro',
    'IGNORANZA EVERYWHERE - PRIMAVERA': 'mario',
    'SOROS FC - PRIMAVERA': 'musci8',
    'MAINZ NA GIOIA - PRIMAVERA': 'franky',
    'PALLA PAZZA - PRIMAVERA': 'nanni',
    'I DISEREDATI - PRIMAVERA': 'emiliano',
    'XYZ - PRIMAVERA': 'luca'
    }
def stats_by_team(stagione ='2020-21', dic = dict_names, primavera = False, personal = False):
    if primavera:
        Rose = rose_complete() 
    else:
        Rose = rose()
    stats = scarica_stats(stagione)
    nomi = list(stats.Nome)
    stats.index = nomi
    
    quot = scarica_quot()
    nomi_Q = list(quot.Nome)
    quot.index = nomi_Q
    
    stats_teams = []
    for team, df in Rose.items():
        for name in df:
            if name is None:
                continue
            elif len(name)<2:
                continue
            if personal:
                classe, eta = personal_info(name, stats)
                temp = list(stats.T[name]) + list(quot.T[name][4:7]) + [classe, eta] +[team, dic[team]]
                col = list(stats.columns) + list(quot.columns)[4:7] + ['Classe', 'Eta\''] +['Nome Squadra', 'Allenatore']
            else:
                temp = list(stats.T[name]) + list(quot.T[name][4:7]) + [team, dic[team]]
                col = list(stats.columns) + list(quot.columns)[4:7] +['Nome Squadra', 'Allenatore']
            stats_teams.append(temp)
    return pd.DataFrame(data= stats_teams, columns = col)

In [19]:
ST = stats_by_team(primavera=True, personal=False)

In [22]:
ST = stats_by_team(primavera = True)
ST

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,...,Ass,Asf,Amm,Esp,Au,Qt. A,Qt. I,Diff.,Nome Squadra,Allenatore
0,2792,P,MUSSO,Udinese,4,5.88,4.38,0,6,0,...,0,0,0,0,0,10,11,-1,PDG 1908,pietro
1,1917,P,DRAGOWSKI,Fiorentina,6,6.25,4.25,0,12,0,...,0,0,0,0,0,10,12,-2,PDG 1908,pietro
2,4957,P,MONTIPO',Benevento,5,5.60,2.80,0,14,0,...,0,0,0,0,0,6,7,-1,PDG 1908,pietro
3,1852,D,SPINAZZOLA,Roma,6,6.42,6.92,1,0,0,...,0,0,0,0,0,11,10,1,PDG 1908,pietro
4,798,D,SKRINIAR,Inter,2,6.00,6.00,0,0,0,...,0,0,0,0,0,7,8,-1,PDG 1908,pietro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2195,A,OUNAS,Cagliari,3,6.25,6.25,0,0,0,...,0,0,0,0,0,5,5,0,IGNORANZA EVERYWHERE - PRIMAVERA,mario
268,4324,A,OKAKA,Udinese,6,5.83,6.92,2,0,0,...,1,0,1,0,0,16,15,1,IGNORANZA EVERYWHERE - PRIMAVERA,mario
269,475,A,IAGO FALQUE,Benevento,2,5.25,5.25,0,0,0,...,0,0,0,0,0,12,14,-2,IGNORANZA EVERYWHERE - PRIMAVERA,mario
270,2155,A,CUTRONE,Fiorentina,6,5.67,5.67,0,0,0,...,0,0,0,0,0,11,12,-1,IGNORANZA EVERYWHERE - PRIMAVERA,mario


In [23]:
gioc = ST[ST.Nome=='MARTINEZ L.']

In [24]:
name = list(gioc['Nome'])[0]
name = name.replace(' ','-').lower()
Id = str(list(gioc['Id'])[0])
team = list(gioc['Squadra'])[0].lower()

In [25]:
name

'martinez-l.'

In [26]:
Id

'2764'

In [27]:
team

'inter'

In [28]:
link = 'https://www.fantacalcio.it/squadre/'+team+'/'+name+'/'+Id

In [29]:
link

'https://www.fantacalcio.it/squadre/inter/martinez-l./2764'

In [41]:
def personal_info(Name, database = stats_by_team(primavera = True)):
    gioc = database[database.Nome == Name]
    name = Name.replace(' ','-').lower()
    Id = str(list(gioc['Id'])[0])
    team = list(gioc['Squadra'])[0].lower()
    link = 'https://www.fantacalcio.it/squadre/'+team+'/'+name+'/'+Id
    driver.get(link)
    string = driver.find_element_by_xpath("/html/body/div[7]/div[5]/main/div/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/ul[1]/li[3]")
    classe = string.text[16:26]
    eta = string.text[28:30]
    
    return classe, eta


In [43]:
personal_info('MUSSO', scarica_stats())

('06/05/1994', '26')

In [35]:
gioc = ST[ST.Nome == 'MARTINEZ L.']
Name = list(gioc['Nome'])[0]
name = Name.replace(' ','-').lower()
Id = str(list(gioc['Id'])[0])
team = list(gioc['Squadra'])[0].lower()
link = 'https://www.fantacalcio.it/squadre/'+team+'/'+name+'/'+Id
driver.get(link)
string = driver.find_element_by_xpath("/html/body/div[7]/div[5]/main/div/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/ul[1]")

In [38]:
full_name = string.text[3:5]

In [40]:
string.text

'Nome\nMartinez Lautaro\nNazionalità\nArgentina\nData di nascita\n22/08/1997 (23)\nAltezza\n174 cm\nPeso\nn.d.'